In [2]:
import scipy.io as sio
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D

In [3]:
import make_video
import save_images

### Loading data from file

In [4]:
interval = [155.0, 175.0]

casual rotation

In [5]:
interval_rot = [157.0, 167.0]

freaquent moving up and down

In [6]:
interval_fluct = [167.0, 175.0]

In [7]:
original = sio.loadmat('37000_SPD16x16.mat')
sign_matrix = np.rot90(original.get("sign_bb"), 2)
data = original.get("Data")
dt = data[0][1][0][0] * 1.0e-3
t_start = data[1][1][0][0]
t_end = t_start + sign_matrix.shape[2] * dt
t_s = int((interval_rot[0] - t_start) / dt)
t_m = int((interval_rot[1] - t_start) / dt)
t_e = int((interval_fluct[1] - t_start) / dt)
t_rot = slice(t_s, t_m)
t_fluct = slice(t_m, t_e)

По видео можно выделить два временных интервала: на интервале [157.0, 167.0] чётко прослеживается вращение системы, в то время как на интервале [167.0, 175.0] движение системы носит скорее колебательный характер, ввиду чего имеет смысл рассматривать эти два временных интервала по отдельности.

In [8]:
cur_data_rot = sign_matrix[:, :, t_rot]
cur_data_fluct = sign_matrix[:, :, t_fluct]
window = int(1 / dt)

Для облегчения анализа движения системы имеет смысл анализировать движение характерной точки системы, которая бы выражала общее поведение системы. Самым очевидным и простым выбором такой точки является центр массы системы, который будем вычислять следующим образом.

In [9]:
def center_mass_point(matrix):
    sum_x = 0
    sum_y = 0
    s = matrix.sum()
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            sum_x += matrix[i][j]*j
            sum_y += matrix[i][j]*i
    x = sum_x/s
    y = sum_y/s
    return np.round(x), np.round(y)

проанализируем как изменяется положения центра масс системы на обоих временных интервалах

In [10]:
def center_mass_movement(cur_data, interval):
    t = []
    center_mass_coords = []
    inds_of_diffs = []
    i = 0
    prev_center = False
    while i < cur_data.shape[2]:
        center_mass = center_mass_point(cur_data[:, :, i])
        center_mass_coords.append(center_mass)
        t.append(interval[0] + i * dt)
        if prev_center and (center_mass != prev_center):
            inds_of_diffs.append((prev_center, center_mass, i))
        prev_center = center_mass
        i += 1
    return center_mass_coords, inds_of_diffs, t

In [11]:
center_mass_coords_rot, inds_of_diffs_rot, t_rot = center_mass_movement(cur_data_rot, interval_rot)
center_mass_coords_fluct, inds_of_diffs_fluct, t_fluct = center_mass_movement(cur_data_fluct, interval_fluct)

Рассмотрим саначала интервал колебаний. Из данных по изменению положения центра масс видно, что сами колебания так же происходят интервалами. При чем центр масс принимает только два значения: (7.0, 8.0), (8.0, 8.0) (Значение центра массы - (8.0, 7.0) связано с неточностью установления временного диапазона колебаний системы, так как после отбрасывания коротких начальных колебаний данное значение больше не встречается). В целом по полученным данным можно делать выводы о частоте колебаний. 

In [12]:
center_mass_coords_fluct_set = set(center_mass_coords_fluct)
center_mass_coords_fluct_set 

{(8.0, 8.0), (7.0, 8.0), (8.0, 7.0)}

In [14]:
t_fluct[314]

167.5024

In [15]:
center_mass_coords_fluct_set = set(center_mass_coords_fluct[314:])
center_mass_coords_fluct_set

{(8.0, 8.0), (7.0, 8.0)}

Рассмотрим интервал вращения системы.

In [12]:
center_mass_coords_rot_set = set(center_mass_coords_rot)
center_mass_coords_rot_set

{(7.0, 7.0), (8.0, 9.0), (8.0, 8.0), (8.0, 7.0), (7.0, 8.0), (7.0, 9.0)}

In [13]:
def printMatrix(s):
    # Do heading
    print("     ", end="")
    
    for j in range(len(s[0])):
        print("%5d " % j, end="")
    print()
    print("     ", end="")
    for j in range(len(s[0])):
        print("------", end="")
    print()
    # Matrix contents
    for i in range(len(s)):
        print("%3d |" % (i), end="") # Row nums
        for j in range(len(s[0])):
            print("%5d " % (s[i][j]), end="")
        print()

Как видно из визуализации, у положения центра масс давольно небольшой диапазон изменений. Конечно из полученных данных можно делать определённые выводы, о значении частоты вращения, так как имеется цикличность в движении центра масс, но необходимо также рассмотреть варианты улучшить исходные данные, с целью сделать процесс движения системы более выраженным, чтобы наблюдения по одной точке ближе соответствовали визуализации системы.

Рассмотрим состояния системы в моменты времени t1 и t2, с разным положением центра массы. 

(7.0, 9.0)

In [14]:
printMatrix(cur_data_rot[:, :, 3255])

         0     1     2     3     4     5     6     7     8     9    10    11    12    13    14    15 
     ------------------------------------------------------------------------------------------------
  0 |   97   139   239   257   295   333   325   326   351   449   254   286   301   293   252   238 
  1 |    5   162   207   233   240   250   285   275   310   129   251   270   258   248   230   218 
  2 |   10   153   186   221   236   266   281   290   313   270   252   236   249   213   200   206 
  3 |  117   186   227   271   298   332   354   357   366   330   298   270   272   235   217   231 
  4 |  126   204   250   305   304   364   398   383   376   384   312   269   270   219   189   226 
  5 |  201   256   363   397   440   477   511   524   482   432   390   358   326   285   271   269 
  6 |  234   302   445   508   535   560   588   600   540   488   459   426   365   338   314   308 
  7 |  267   354   557   636   700   699   702   701   661   625   552   513   449

(8.0, 8.0)

In [15]:
printMatrix(cur_data_rot[:, :, 3310])

         0     1     2     3     4     5     6     7     8     9    10    11    12    13    14    15 
     ------------------------------------------------------------------------------------------------
  0 |   76   125   215   243   332   398   438   460   453   628   307   336   348   344   308   291 
  1 |    6   175   230   266   331   340   399   394   430   246   330   344   332   321   304   286 
  2 |   10   151   205   262   332   391   439   446   459   405   368   341   335   300   276   278 
  3 |  149   225   309   357   452   534   630   622   601   539   483   456   448   407   367   369 
  4 |  139   232   303   369   442   609   771   750   658   629   504   467   449   387   343   373 
  5 |  227   293   393   452   607   798  1013  1015   845   735   628   580   536   482   452   452 
  6 |  251   328   410   482   667   931  1188  1152   880   773   695   646   593   529   493   486 
  7 |  252   324   424   483   711   982  1303  1226   941   845   720   663   610

In [14]:
print(f'number of centers position changes in rotation fase = {len(inds_of_diffs_rot)}')
print(f'number of time intervals in rotation fase = {len(t_rot)}')
print(f'number of centers position changes in fluctuation fase = {len(inds_of_diffs_fluct)}')
print(f'number of time intervals in fluctuation fase = {len(t_fluct)}')

number of centers position changes in rotation fase = 248
number of time intervals in rotation fase = 6250
number of centers position changes in fluctuation fase = 106
number of time intervals in fluctuation fase = 5000


#### Измерение частоты вращения

Для вычисления частоты будем использовать формулу $\nu = \frac{1}{T}$. Для разных фаз системы период будем считать по разному: для фазы колебаний период - время, за которое центр масс отклоняется и возрвращается в исходную точку, для фазы вращения - время с окончания последнего оборота и до возвращения центра масс в стартовую точку поменяв свое положение больше 1 раза ( в фазе вращения мы чётко видим основной цикл вращения, и таким образом, игнорируя небольшие погрешности в положении центра масс, сможем получить более достоверные результаты). 

In [15]:
def frequency(inds_of_diffs, t, fluct=False, fluct_count_limit=1):
    cycle_frequency = 0
    frequency_per_time = []
    idx_start = 0
    period = 0
    periods = []
    if fluct:
        for i in range(1, len(inds_of_diffs), 2):
            period = t[inds_of_diffs[i][2]] - t[idx_start]
            cycle_frequency = 1/period
            periods.append(period)
            for k in range(0, (inds_of_diffs[i][2] - idx_start)):
                frequency_per_time.append(cycle_frequency)
            idx_start = inds_of_diffs[i][2]
    else:
        step = 1
        fluct_count = 0
        pos_start = 0
        temp_idx_start = 0
        temp_pos_start = 0
        temp_fluct_freq = []
        temp_periods = []
        for i in range(1, len(inds_of_diffs), 1):
            step += 1
            if (step == 2) and (inds_of_diffs[i][1] == inds_of_diffs[i-1][0]):
                period = t[inds_of_diffs[i][2]] - t[temp_idx_start]
                cycle_frequency = 1/period
                temp_periods.append(period)
                for k in range(0, (inds_of_diffs[i][2] - temp_idx_start)):
                    temp_fluct_freq.append(cycle_frequency)
                temp_idx_start = inds_of_diffs[i][2]
                temp_pos_start = i
                fluct_count += 1
                continue
            if step == 3:
                if fluct_count >= fluct_count_limit:
                    frequency_per_time.extend(temp_fluct_freq)
                    periods.extend(temp_periods)
                    pos_start = temp_pos_start
                    idx_start = temp_idx_start
                    step = 1
                fluct_count = 0
                temp_fluct_freq = []
                temp_periods = []
            if step >= 3:
                for s in range(pos_start,i-1):
                    if inds_of_diffs[i][1] == inds_of_diffs[s][1]:
                        period = t[inds_of_diffs[i][2]] - t[temp_idx_start]
                        cycle_frequency = 1/period
                        periods.append(period)
                        for k in range(0, (inds_of_diffs[i][2] - idx_start)):
                            frequency_per_time.append(cycle_frequency)
                        idx_start = inds_of_diffs[i][2]
                        temp_idx_start = idx_start
                        pos_start = i
                        temp_pos_start = pos_start
                        step = 0
                        break
    return frequency_per_time, periods

In [16]:
freq_rot, rot_periods = frequency(inds_of_diffs_rot, t_rot, 3)
last_freq = freq_rot[-1]
for k in range(0, (len(t_rot) - len(freq_rot))):
    freq_rot.append(last_freq)

In [17]:
plt.figure(figsize=(20,10))
plt.xlabel('time')
plt.ylabel('frequency')
plt.title('Frequency in rotation interval')
plt.plot(t_rot, freq_rot, ms=1.5)    
plt.show()

In [18]:
freq_fluct, fluct_periods = frequency(inds_of_diffs_fluct, t_fluct, True)
len(freq_fluct)

4827

In [46]:
plt.figure(figsize=(20,10))
plt.xlabel('time')
plt.ylabel('frequency')
plt.title('Frequency in fluctuation interval')
plt.plot(t_fluct[:4827], freq_fluct, ms=1.5)    
plt.show()

In [19]:
freq_rot = np.array(freq_rot)
freq_fluct = np.array(freq_fluct)

Выборочные характеристики по частоте

In [43]:
print(f"mean in rotation fase = {np.mean(freq_rot)}")
print(f"median in rotation fase = {np.median(freq_rot)}")
print(f"mean in fluctuation fase = {np.mean(freq_fluct)}")
print(f"median in fluctuation fase = {np.median(freq_fluct)}")

mean in rotation fase = 12.928571428572038
median in rotation fase = 5.48245614035084
mean in fluctuation fase = 6.86244043919688
median in fluctuation fase = 1.0557432432432328


In [21]:
from scipy import stats

In [22]:
print(stats.pearsonr(t_rot,freq_rot))

(0.1461418970563667, 3.514105639086008e-31)


Из рассмотрения данных вытекают два возможных направления улучшения данных: 1. Фильтровка - бросается в глаза высокая доля "шумовых" значений - значений, которые практически не меняются при изменении положения системы, но вносят существенный вклад в положение центра масс. 2. Ослабление ядра - из визуализации и по рассмотренным матрицам видно, что на интервале вращения система имеет форму ядра с хвостом (по которому и прослеживается вращение). При такой структуре, высокий вес ядра, относительно хвоста, в случае низкой дискретизации пространства, может приводить к эффекту статичности центра масс. Для того чтобы исключить подобный случай, можно ослабить ядерную часть системы, удалив из неё ряд элементов. Это приведёт к смещению центра масс в сторону хвоста и должно повысить вариантивность значений его положения.

#### Фильтровка

чтобы понять как лучше произвести фильтрацию, рассмотрим ряд выборочных характеристик положения и разброса по значениям в матрице системы. Рассмотрим следующие характеристики: медиана, квартили (1 и 3 четвёртых), среднее стандартное отклонение. 

In [23]:
matrics_rot = []
for i in range(cur_data_rot.shape[2]):
    matrics_rot.append(cur_data_rot[:, :, i])

matrics_fluct = []
for i in range(cur_data_fluct.shape[2]):
    matrics_fluct.append(cur_data_fluct[:, :, i])

In [25]:
matrics_rot_perc_25 = np.mean([np.percentile(mat.flatten(),25) for mat in matrics_rot])
matrics_fluct_perc_25 = np.mean([np.percentile(mat.flatten(),25) for mat in matrics_fluct])

print(f'percentile 25 rot = {matrics_rot_perc_25}')
print(f'percentile 25 fluct = {matrics_fluct_perc_25}')

percentile 25 rot = 395.1756
percentile 25 fluct = 186.31995


In [26]:
filtered_data_rot = np.copy(cur_data_rot)
for k in range(filtered_data_rot.shape[2]):
    for j in range(filtered_data_rot.shape[1]):
        for i in range(filtered_data_rot.shape[0]):
            if filtered_data_rot[i,j,k] <= matrics_rot_perc_25:
                filtered_data_rot[i,j,k] = 0

In [27]:
filtered_data_fluct = np.copy(cur_data_fluct)
for k in range(filtered_data_fluct.shape[2]):
    for j in range(filtered_data_fluct.shape[1]):
        for i in range(filtered_data_fluct.shape[0]):
            if filtered_data_fluct[i,j,k] <= matrics_fluct_perc_25:
                filtered_data_fluct[i,j,k] = 0

In [23]:
matrics_median = np.mean([np.median(mat.flatten()) for mat in matrics])
matrics_perc_25 = np.mean([np.percentile(mat.flatten(),25) for mat in matrics])
matrics_perc_75 = np.mean([np.percentile(mat.flatten(),75) for mat in matrics])
matrics_std = np.mean([np.std(mat.flatten()) for mat in matrics])
print(f'median = {matrics_median}')
print(f'percentile 25 = {matrics_perc_25}')
print(f'percentile 75 = {matrics_perc_75}')
print(f'std = {matrics_std}')

median = 538.1099243164062
percentile 25 = 395.1756
percentile 75 = 748.40524
std = 282.3274841308594


In [28]:
def drop_corn_element(data, drops_num):
    droped_corn_data = np.copy(data)
    for k in range(droped_corn_data.shape[2]):  
        for i in range(0,drops_num):
            max_pos = droped_corn_data[:,:,k].argmax()
            droped_corn_data[:,:,k][max_pos//16][max_pos%16] = 0 
    return droped_corn_data    

In [29]:
droped_corn_data_rot = drop_corn_element(filtered_data_rot, 2)
droped_corn_data_fluct = drop_corn_element(filtered_data_fluct, 2)

Рассмотрим как выглядит поведение системы если отфильтровать все значения ниже и равные 25 персентилю и удалить 2 элемента из ядра системы. Визуализируем это.

In [79]:
save_images.save_projections_for_video(droped_corn_data_rot, t_rot[0], dt, 2, 'pictures_full_rot')

In [22]:
make_video.generate_video(pathIn = "C:\\Users\\Dmitrii\\Python Projects\\Mat Stat Course project\\pictures_full_rot\\",
                          pathOut="projections_dropped_rot.avi", fps=10)

In [23]:
save_images.save_projections_for_video(droped_corn_data_fluct, t_rot[0], dt, 2, 'pictures_full_fluct')
make_video.generate_video(pathIn = "C:\\Users\\Dmitrii\\Python Projects\\Mat Stat Course project\\pictures_full_fluct\\",
                          pathOut="projections_dropped_fluct.avi", fps=10)

In [27]:
make_video.generate_video(pathIn = "C:\\Users\\Dmitrii\\Python Projects\\Mat Stat Course project\\pictures_full\\",
                          pathOut="centers_mass_filtered.avi", fps=5)

In [30]:
center_mass_coords_rot_droped, inds_of_diffs_rot_droped, t_rot_droped = center_mass_movement(droped_corn_data_rot, interval_rot)
center_mass_coords_fluct_droped, inds_of_diffs_fluct_droped, t_fluct_droped = center_mass_movement(droped_corn_data_fluct, interval_fluct)

In [22]:
center_mass_coords_rot_set = set(center_mass_coords_rot)
center_mass_coords_rot_set

{(7.0, 7.0), (7.0, 8.0), (7.0, 9.0), (8.0, 7.0), (8.0, 8.0), (8.0, 9.0)}

In [24]:
center_mass_coords_rot_droped_set = set(center_mass_coords_rot_droped)
center_mass_coords_rot_droped_set

{(6.0, 8.0),
 (6.0, 9.0),
 (7.0, 7.0),
 (7.0, 8.0),
 (7.0, 9.0),
 (7.0, 10.0),
 (8.0, 6.0),
 (8.0, 7.0),
 (8.0, 8.0),
 (8.0, 9.0),
 (8.0, 10.0),
 (9.0, 6.0),
 (9.0, 7.0),
 (9.0, 8.0),
 (9.0, 9.0)}

Видим, что разнообразие положений центра масс сильно повысилось.

In [27]:
center_mass_coords_fluct_droped_set = set(center_mass_coords_fluct_droped)
center_mass_coords_fluct_droped_set

{(7.0, 8.0), (7.0, 9.0), (7.0, 10.0), (8.0, 8.0), (8.0, 9.0), (8.0, 10.0)}

In [31]:
freq_rot_droped, rot_periods_droped = frequency(inds_of_diffs_rot_droped, t_rot, 3)
last_freq = freq_rot_droped[-1]
for k in range(0, (len(t_rot) - len(freq_rot_droped))):
    freq_rot_droped.append(last_freq)

In [32]:
plt.figure(figsize=(20,10))
plt.xlabel('time')
plt.ylabel('frequency')
plt.title('Frequency in rotation interval')
plt.plot(t_rot, freq_rot_droped, ms=1.5)    
plt.show()

In [51]:
freq_fluct_droped, fluct_periods_droped = frequency(inds_of_diffs_fluct_droped, t_fluct, 1)
len(freq_fluct_droped)

4979

In [52]:
len(freq_fluct_droped)

4979

In [53]:
plt.figure(figsize=(20,10))
plt.xlabel('time')
plt.ylabel('frequency')
plt.title('Frequency in fluctuation interval')
plt.plot(t_fluct[:4979], freq_fluct_droped, ms=1.5)    
plt.show()

In [44]:
freq_rot_droped = np.array(freq_rot_droped)
freq_fluct_droped = np.array(freq_fluct_droped)
print(f"mean in rotation fase = {np.mean(freq_rot_droped)}")
print(f"median in rotation fase = {np.median(freq_rot_droped)}")
print(f"mean in fluctuation fase = {np.mean(freq_fluct_droped)}")
print(f"median in fluctuation fase = {np.median(freq_fluct_droped)}")
print(stats.pearsonr(t_rot,freq_rot_droped))
print(stats.pearsonr(t_fluct[:4979],freq_fluct_droped))

mean in rotation fase = 26.350000000002407
median in rotation fase = 15.625000000001721
mean in fluctuation fase = 48.830086362721204
median in fluctuation fase = 10.96491228070339
(0.28230234970606005, 7.516049282282882e-115)
(0.43133187020735275, 8.798898724548912e-225)


Рассмотрим случай сильной фильтрации. Удалим из рассмотрения все зоны светимости со значением меньше 75 персентиля. Ядро ослаблять не будем.

In [35]:
matrics_rot_perc_75 = np.mean([np.percentile(mat.flatten(),75) for mat in matrics_rot])
matrics_fluct_perc_75 = np.mean([np.percentile(mat.flatten(),75) for mat in matrics_fluct])

print(f'percentile 25 rot = {matrics_rot_perc_75}')
print(f'percentile 25 fluct = {matrics_fluct_perc_75}')

percentile 25 rot = 748.40524
percentile 25 fluct = 281.14925


In [36]:
high_filtered_data_rot = np.copy(cur_data_rot)
for k in range(high_filtered_data_rot.shape[2]):
    for j in range(high_filtered_data_rot.shape[1]):
        for i in range(high_filtered_data_rot.shape[0]):
            if high_filtered_data_rot[i,j,k] <= matrics_rot_perc_75:
                high_filtered_data_rot[i,j,k] = 0

In [37]:
high_filtered_data_fluct = np.copy(cur_data_fluct)
for k in range(high_filtered_data_fluct.shape[2]):
    for j in range(high_filtered_data_fluct.shape[1]):
        for i in range(high_filtered_data_fluct.shape[0]):
            if high_filtered_data_fluct[i,j,k] <= matrics_fluct_perc_75:
                high_filtered_data_fluct[i,j,k] = 0

In [38]:
center_mass_coords_rot_filtered, inds_of_diffs_rot_filtered, t_rot_filtered = center_mass_movement(high_filtered_data_rot, interval_rot)
center_mass_coords_fluct_filtered, inds_of_diffs_fluct_filtered, t_fluct_filtered = center_mass_movement(high_filtered_data_fluct, interval_fluct)

__main__:9: RuntimeWarning: invalid value encountered in double_scalars
__main__:10: RuntimeWarning: invalid value encountered in double_scalars


In [39]:
freq_rot_filtered, rot_periods_filtered = frequency(inds_of_diffs_rot_filtered, t_rot, 3)
last_freq = freq_rot_droped[-1]
for k in range(0, (len(t_rot) - len(freq_rot_filtered))):
    freq_rot_filtered.append(last_freq)

In [40]:
plt.figure(figsize=(20,10))
plt.xlabel('time')
plt.ylabel('frequency')
plt.title('Frequency in rotation interval')
plt.plot(t_rot, freq_rot_filtered, ms=1.5)    
plt.show()

In [41]:
freq_fluct_filtered, fluct_periods_filtered = frequency(inds_of_diffs_fluct_filtered, t_fluct, 1)
len(freq_fluct_filtered)

4998

In [54]:
plt.figure(figsize=(20,10))
plt.xlabel('time')
plt.ylabel('frequency')
plt.title('Frequency in fluctuation interval')
plt.plot(t_fluct[:4998], freq_fluct_filtered, ms=1.5)    
plt.show()

In [45]:
freq_rot_droped = np.array(freq_rot_filtered)
freq_fluct_droped = np.array(freq_fluct_filtered)
print(f"mean in rotation fase = {np.mean(freq_rot_filtered)}")
print(f"median in rotation fase = {np.median(freq_rot_filtered)}")
print(f"mean in fluctuation fase = {np.mean(freq_fluct_filtered)}")
print(f"median in fluctuation fase = {np.median(freq_fluct_filtered)}")
print(stats.pearsonr(t_rot,freq_rot_filtered))
print(stats.pearsonr(t_fluct[:4998],freq_fluct_filtered))

mean in rotation fase = 46.10000000000464
median in rotation fase = 32.894736842089664
mean in fluctuation fase = 78.65646258503712
median in fluctuation fase = 22.321428571438112
(0.25032441988432447, 6.42484560932837e-90)
(0.618712230185287, 0.0)
